# Geographic Visualizations with Maps

## Interactive Maps using Folium and Plotly
This notebook creates geographic visualizations of transportation inequity data.

In [ ]:
import pandas as pd
import geopandas as gpd
import folium
import plotly.express as px

# Load cleaned data
df = pd.read_csv('../data/cleaned/clean_transportation.csv')
print(df.head())

In [ ]:
# Create an interactive Plotly map (requires geographic data with coordinates)
# Example: Scatter mapbox showing neighborhoods with commute time

fig = px.scatter_mapbox(
    df.groupby(['neighborhood', 'latitude', 'longitude'])['commute_time'].mean().reset_index(),
    lat='latitude',
    lon='longitude',
    color='commute_time',
    size='commute_time',
    hover_name='neighborhood',
    hover_data={'commute_time': ':.1f', 'latitude': ':.4f', 'longitude': ':.4f'},
    color_continuous_scale='RdYlGn_r',  # Red for high, green for low
    zoom=9,
    title='Average Commute Time by Neighborhood'
)

fig.update_layout(
    mapbox_style='open-street-map',
    height=600,
    width=900
)

fig.write_html("../visuals/maps/commute_time_plotly.html")
print("Plotly map saved!")
fig.show()

## 6. Plotly Interactive Map (Choropleth Example)

In [ ]:
# Create map with transportation cost burden
df['transport_cost_burden'] = (df['annual_transport_cost'] / df['income']) * 100

burden_by_neighborhood = df.groupby(['neighborhood', 'latitude', 'longitude'])['transport_cost_burden'].mean().reset_index()

m_burden = folium.Map(location=[df['latitude'].mean(), df['longitude'].mean()], zoom_start=10)

for idx, row in burden_by_neighborhood.iterrows():
    # Color code by burden percentage (green = low burden, red = high burden)
    burden = row['transport_cost_burden']
    if burden < 10:
        color = 'green'
    elif burden < 15:
        color = 'yellow'
    else:
        color = 'red'
    
    folium.CircleMarker(
        location=[row['latitude'], row['longitude']],
        radius=10,
        popup=f"{row['neighborhood']}<br>Transport Cost Burden: {burden:.1f}% of income",
        color=color,
        fill=True,
        fillColor=color,
        fillOpacity=0.7
    ).add_to(m_burden)

m_burden.save("../visuals/maps/cost_burden_map.html")
print("Cost burden map saved!")
m_burden

## 5. Income & Transportation Cost Burden Map

In [ ]:
# Create map with transit distance data
transit_by_neighborhood = df.groupby(['neighborhood', 'latitude', 'longitude'])['distance_to_transit'].mean().reset_index()

m_transit = folium.Map(location=[df['latitude'].mean(), df['longitude'].mean()], zoom_start=10)

for idx, row in transit_by_neighborhood.iterrows():
    # Color code by distance to transit (green = close, red = far)
    distance = row['distance_to_transit']
    if distance < 0.5:
        color = 'green'
    elif distance < 1.0:
        color = 'yellow'
    else:
        color = 'red'
    
    folium.CircleMarker(
        location=[row['latitude'], row['longitude']],
        radius=10,
        popup=f"{row['neighborhood']}<br>Distance to Transit: {distance:.2f} miles",
        color=color,
        fill=True,
        fillColor=color,
        fillOpacity=0.7
    ).add_to(m_transit)

m_transit.save("../visuals/maps/transit_access_map.html")
print("Transit access map saved!")
m_transit

## 4. Transit Access Heatmap

In [ ]:
# Create map with vehicle access data
vehicle_by_neighborhood = df.groupby(['neighborhood', 'latitude', 'longitude'])['vehicle_access'].mean().reset_index()

m_vehicle = folium.Map(location=[df['latitude'].mean(), df['longitude'].mean()], zoom_start=10)

for idx, row in vehicle_by_neighborhood.iterrows():
    # Color code by vehicle ownership rate (green = high, red = low)
    access_rate = row['vehicle_access']
    if access_rate > 0.75:
        color = 'green'
    elif access_rate > 0.50:
        color = 'yellow'
    else:
        color = 'red'
    
    folium.CircleMarker(
        location=[row['latitude'], row['longitude']],
        radius=10,
        popup=f"{row['neighborhood']}<br>Vehicle Access: {access_rate*100:.1f}%",
        color=color,
        fill=True,
        fillColor=color,
        fillOpacity=0.7
    ).add_to(m_vehicle)

m_vehicle.save("../visuals/maps/vehicle_access_map.html")
print("Vehicle access map saved!")
m_vehicle

## 3. Vehicle Access Heatmap

In [ ]:
# Create map with commute time data by neighborhood
# First, get average commute time by neighborhood
commute_by_neighborhood = df.groupby(['neighborhood', 'latitude', 'longitude'])['commute_time'].mean().reset_index()

# Create base map (adjust coordinates to your data region)
m_commute = folium.Map(location=[df['latitude'].mean(), df['longitude'].mean()], zoom_start=10)

# Add markers for each neighborhood
for idx, row in commute_by_neighborhood.iterrows():
    # Color code by commute time (red = longer, green = shorter)
    if row['commute_time'] > 45:
        color = 'red'
    elif row['commute_time'] > 30:
        color = 'orange'
    else:
        color = 'green'
    
    folium.CircleMarker(
        location=[row['latitude'], row['longitude']],
        radius=10,
        popup=f"{row['neighborhood']}<br>Avg Commute: {row['commute_time']:.1f} min",
        color=color,
        fill=True,
        fillColor=color,
        fillOpacity=0.7
    ).add_to(m_commute)

m_commute.save("../visuals/maps/commute_time_map.html")
print("Commute time map saved!")
m_commute

## 2. Commute Time Heatmap by Neighborhood

In [ ]:
# Create a base map of the United States
# Center coordinates: [39, -98] = center of USA
m = folium.Map(location=[39, -98], zoom_start=4, tiles='OpenStreetMap')
m.save("../visuals/maps/base_map.html")
print("Base map saved to ../visuals/maps/base_map.html")
m

## 1. Base Map Creation